In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

In [3]:
from fl.preprocessing import load_mnist, data_to_client
from fl.model import NN
from fl.utils import plot_train_and_test, weights_to_json
from fl.federated_learning import federated

2024-04-26 10:44:09.820197: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-26 10:44:10.006209: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-26 10:44:10.684196: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 10:44:12.043217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Challenge 2 : Du poison [2/2]

![du_poison_2.jpg](https://i.imgflip.com/8nfix9.jpg)

Cette fois, une défense a été mise en place. Il s'agit d'éviter qu'un client ait trop d'influence en forçant une variation maximale pour les poids. Mais *est-ce suffisant ?* 

## Récupération du drapeau 

Comme pour le premier challenge, il faut envoyer ses poids à l'API. 

In [6]:
epochs = 5
epochs_fl = 1
nb_clients = 5

x_train, y_train, x_test, y_test = load_mnist()
x_clients, y_clients = data_to_client(x_train, y_train, nb_clients=nb_clients)


In [7]:
# randomize y_clients and y_test
import numpy as np
np.random.shuffle(y_clients)
np.random.shuffle(y_test)

In [8]:
federated_learning = federated(
    NN(),
    x_clients, 
    y_clients, 
    x_test, 
    y_test, 
    fl_iterations=epochs,
    epochs_fl=epochs_fl
)

Federated learning iteration: 1
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Federated Accuracy: 0.095
Federated learning iteration: 2
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Federated Accuracy: 0.091
Federated learning iteration: 3
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Federated Accuracy: 0.088
Federated learning iteration: 4
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━

In [27]:
test = federated_learning['weights'].copy()
print(test[0][:5])
# multiply all values of test by -1
for i, array in enumerate(test):
    np.multiply(array, -10, out=array)
    test[i] = array
print(test[0][:5])

[[-4648460.5   3697421.5  -4386023.   ...   713548.44 -2290995.5
  -5620294.  ]
 [-4065347.2  -1439451.4   5080609.   ... -4017437.8   2122496.2
  -1072391.1 ]
 [ 2471737.8   -771595.94  3434683.   ...  3524893.8  -4791258.
  -1493425.2 ]
 [ 3274128.8   5771088.    -371401.38 ...  -794018.5  -2107110.2
    570482.4 ]
 [  438160.88  4994144.    -943385.3  ...   -89957.95 -2473497.
   -327283.16]]
[[ 46484604.  -36974216.   43860232.  ...  -7135484.5  22909956.
   56202940. ]
 [ 40653472.   14394514.  -50806088.  ...  40174376.  -21224962.
   10723911. ]
 [-24717378.    7715959.5 -34346832.  ... -35248936.   47912580.
   14934252. ]
 [-32741288.  -57710880.    3714013.8 ...   7940185.   21071102.
   -5704824. ]
 [ -4381609.  -49941440.    9433853.  ...    899579.5  24734970.
    3272831.5]]


In [2]:
model = ...
raise NotImplementedError

In [4]:
import requests as rq

URL = "https://du-poison.challenges.404ctf.fr"
rq.get(URL + "/healthcheck").json()

{'message': 'Statut : en pleine forme !'}

In [25]:
# d = weights_to_json(model.get_weights())
d = weights_to_json(federated_learning['weights'])
d = weights_to_json(test)
rq.post(URL + "/challenges/2", json=d).json()

{'message': "Raté ! Le score de l'apprentissage fédéré est de 0.9365. Il faut l'empoisonner pour qu'il passe en dessous de 0.5"}